In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from runOPAL import OpalRunner, SlurmJob

# Config

In [ ]:
base_name = 'fullBeamline'

# directory that contains the <base_name>.data file and a file tmpl/<base_name>.tmpl
input_directory = '/data/user/bellotti_r/awa_masters/renato/test'

# directory whose subdirectories will contain:
# - <base_name>.stat files
# - <base_name>.json (containing the design variable configuration)
# - <base_name>.in (input file)
output_dir = f'/data/user/bellotti_r/test_output'

# directory where the fieldmaps are stored
fieldmap_dir = f'{input_directory}/fieldmaps'

# Create some test configurations:

In [ ]:
dvars = pd.DataFrame(columns=['IBF', 'IM', 'GPHASE', 'ILS1', 'ILS2', 'ILS3', 'bunchCharge', 'lambda', 'SIGXY'])
dvars.loc[0] = [450, 100., -50, 0, 0, 0, 0.3, 0.3, 1.5]
dvars.loc[1] = [500, 150, 0, 10, 20, 30, 1, 1, 10]

In [ ]:
dvars

# Run the simulations

In [ ]:
runner = OpalRunner(input_directory, output_dir, fieldmap_dir, base_name)

In [ ]:
IDs = runner.run_configurations_blocking(dvars)

# Alternative:
# runner.run_configurations(dvars)

# Evaluate the results

`beams` is callable. Its only parameter is a float $s$ indicating a position. `beams(s)` returns the indicated quantities of interest (here: $\sigma_x, \epsilon_x$) at position $s$.<br />
The rows of the result correspond the the indices of the design variable configurations that were run before.

In [ ]:
beams = runner.get_quantities_of_interest(['RMS Beamsize in x', 'Normalized Emittance x'],
                                          [0, 1])

In [ ]:
beams(0.1)

In [ ]:
beams(0.01)

In [ ]:
s_values = np.linspace(0, 0.3, 50)

sigma_x = pd.concat([beams(s)['RMS Beamsize in x'] for s in s_values], axis=1).T.reset_index(drop=True)
epsilon_x = pd.concat([beams(s)['Normalized Emittance x'] for s in s_values], axis=1).T.reset_index(drop=True)

# Sanity check: Compare to stat file

In [ ]:
# compare to .stat file
from mllib.data.opal_stat_file_to_dataframe import StatFile
df = StatFile('/data/user/bellotti_r/test_output/0/fullBeamline.stat').getDataFrame()

In [ ]:
fig, axes = plt.subplots(1, 2)

ax = axes[0]
ax.plot(s_values, sigma_x[0], label='OpalRunner')
ax.plot(df['Path length'], df['RMS Beamsize in x'], label='StatFile')
ax.legend()
ax.set_xlabel('Path length [m]')
ax.set_ylabel('$\sigma_x$ [m]');

ax = axes[1]
ax.plot(s_values, epsilon_x[0], label='OpalRunner')
ax.plot(df['Path length'], df['Normalized Emittance x'], label='StatFile')
ax.legend()
ax.set_xlabel('Path length [m]')
ax.set_ylabel('$\epsilon_x$ [m rad]');

fig.tight_layout()